In [76]:
import akshare as ak
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import datetime as dt

In [62]:
stock_data_pool = ak.stock_zh_a_spot_em() 
stock_data_pool.dropna(inplace=True)
stock_data_pool['总市值'] = (pd.to_numeric(stock_data_pool['总市值']) / 100000000).round(2)
print(stock_data_pool)

  0%|          | 0/57 [00:00<?, ?it/s]

        序号      代码    名称   最新价    涨跌幅   涨跌额        成交量           成交额    振幅  \
1        2  837748  路桥信息 57.74  29.99 13.32   87361.00  416384564.04 34.20   
2        3  300368  汇金股份 10.36  20.05  1.73 1578797.00 1529114306.01 17.84   
3        4  300513  恒实科技  9.40  20.05  1.57  397558.00  365352788.10 10.98   
4        5  300082  奥克股份  9.53  20.03  1.59 1204287.00 1097338585.64 18.51   
5        6  300651  金陵体育 29.38  20.02  4.90  271744.00  760886919.41 15.52   
...    ...     ...   ...   ...    ...   ...        ...           ...   ...   
5384  5385  000026   飞亚达 13.32 -10.00 -1.48   88140.00  117894776.41  4.12   
5385  5386  001332  锡装股份 37.08 -10.00 -4.12   79002.00  302176574.06  5.49   
5386  5387  688648  中邮科技 54.00 -12.76 -7.90  136016.00  756039072.00 16.56   
5387  5388  300945   曼卡龙 25.05 -14.71 -4.32  614304.00 1626695093.62 14.10   
5388  5389  300119  瑞普生物 22.41 -14.82 -3.90  544557.00 1276027575.61 16.65   

        最高  ...   量比   换手率  市盈率-动态   市净率    总市值          流通市值  

In [64]:
#筛选出涨幅在3%~5%之间的股票 
first_filtered_data = stock_data_pool[(stock_data_pool['涨跌幅'] > 3) & (stock_data_pool['涨跌幅'] < 5)]
#筛选出换手率在4%~10%之间的股票
first_filtered_data = first_filtered_data[(first_filtered_data['换手率'] > 4) & (first_filtered_data['换手率'] < 10)]
#删除量比小于1的股票
first_filtered_data = first_filtered_data[first_filtered_data['量比'] > 1]
# 选出市值在50亿到100亿之间的股票
first_filtered_data = first_filtered_data[(first_filtered_data['总市值'] > 50) & (first_filtered_data['总市值'] < 100)]

# file_path = f'e:\\desktop\\stock.csv'
# data.to_csv(f'e:\\desktop\\stock.csv', index=False,encoding ='utf-8-sig')
print(len(first_filtered_data))

52


In [78]:
today = dt.datetime.now().strftime('%Y%m%d')
start_date = (dt.datetime.now() - dt.timedelta(days=60)).strftime('%Y%m%d')
ffdc_stock_data_test = ak.stock_zh_a_hist(symbol="600900", 
                                        period = "daily", 
                                        start_date = start_date,  
                                        end_date =  today, 
                                        adjust="qfq")
print(ffdc_stock_data_test.columns)

Index(['日期', '股票代码', '开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额',
       '换手率'],
      dtype='object')


In [ ]:
first_filtered_data_codes = first_filtered_data['代码'].tolist()
today = dt.datetime.now().strftime('%Y%m%d')
start_date = (dt.datetime.now() - dt.timedelta(days=60)).strftime('%Y%m%d')

print(first_filtered_data_codes)

for ffdc in first_filtered_data_codes[:3]:
    ffdc_stock_data = ak.stock_zh_a_hist(symbol=ffdc, 
                                         period = "daily", 
                                         start_date = start_date,  
                                         end_date =  today, 
                                         adjust="hfq")
    
    ma5 = ffdc_stock_data['收盘'].rolling(window=5).mean()
    ma20 = ffdc_stock_data['收盘'].rolling(window=20).mean()
    
    latest_ma5 = ma5.iloc[-1]
    latest_ma20 = ma20.iloc[-1]
    
    print(latest_ma5)
    
    if len(ma5) >= 5 and len(ma20) >= 20:
        ma5_slope = (ma5.iloc[-1] - ma5.iloc[-5]) / 5
        ma20_slope = (ma20.iloc[-1] - ma20.iloc[-5]) / 5
    else:
        ma5_slope = 0
        ma20_slope = 0
        
    is_down = (latest_ma5 < latest_ma20) and (ma5_slope < 0 or ma20_slope < 0)
    
    # if is_down:
    #     print(f"股票代码: {ffdc}, 处于下降趋势")
    # else:
    #     print(f"股票代码: {ffdc}, 不处于下降趋势")
    
    print(f"股票代码: {ffdc}, 是否满足条件: {is_down}")
       
    # file_path = f'e:\\desktop\\ffdc{ffdc}.csv'
    # ffdc_stock_data.to_csv(file_path, index=False,encoding ='utf-8-sig')



['300862', '002046', '688518', '002362', '688320', '002609', '002042', '300895', '300287', '688195', '300053', '002121', '600366', '300579', '600345', '301162', '603516', '300681', '688531', '002843', '300352', '688661', '688173', '603985', '002536', '300354', '300129', '300790', '300643', '002730', '600797', '301628', '600110', '300684', '600590', '688668', '688321', '301191', '300249', '603666', '300680', '301330', '002446', '300245', '301382', '300825', '300814', '300310', '300403', '002380', '301413', '300342']
39.46
股票代码: 300862, 处于下降趋势
70.55
股票代码: 002046, 不处于下降趋势
17.04
股票代码: 688518, 不处于下降趋势
43.53
股票代码: 002362, 不处于下降趋势
43.07
股票代码: 688320, 处于下降趋势


In [ ]:
high = first_filtered_data_codes.row['最高']
low = first_filtered_data_codes.row['最低']
open_price = first_filtered_data_codes.row['开盘']
close_price = first_filtered_data_codes.row['收盘']
volume = first_filtered_data_codes.row['成交量']
total_range = high - low
